In [2]:
import pandas as pd

df = pd.read_csv('./data/post_transform.csv')
df.head(5)

,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,applicant_ethnicity,...,property_type_str,loan_purpose_str,occupancy_str,preapproval_str,applicant_ethnicity_str,applicant_race_str,applicant_sex_str,msa_md_str,state_code_str,county_code_str
0,3,1,1,1,70.0,3,18,37,246,2,...,1 to 4 Family - non manufactured,Home purchase,Owner Occupied,NaN,Not Hispanic,White,Male,18.0,37.0,246.0
1,1,1,3,1,178.0,3,369,52,299,1,...,1 to 4 Family - non manufactured,Refinancing,Owner Occupied,NaN,Hispanic,White,Male,369.0,52.0,299.0
2,2,1,3,1,163.0,3,16,10,306,2,...,1 to 4 Family - non manufactured,Refinancing,Owner Occupied,NaN,Not Hispanic,White,Male,16.0,10.0,306.0
3,1,1,1,1,155.0,1,305,47,180,2,...,1 to 4 Family - non manufactured,Home purchase,Owner Occupied,Requested,Not Hispanic,White,Male,305.0,47.0,180.0
4,1,1,1,1,305.0,3,24,37,20,2,...,1 to 4 Family - non manufactured,Home purchase,Owner Occupied,NaN,Not Hispanic,African American,Female,24.0,37.0,20.0


In [3]:
# scale the numeric columns 

from sklearn.preprocessing import RobustScaler

# float columns to consider
numeric_cols =[ 'loan_amount', 'applicant_income', 'population', 'ffiecmedian_family_income',
               'number_of_owner-occupied_units', 'number_of_1_to_4_family_units']


# use robust scaler for outlier numerics
for col in numeric_cols:
 #   utilities.minMax_normalize_df_col(df, col)
    col_copy = df[[ col ]].copy()
    transformer = RobustScaler().fit( col_copy  )
    df[ col ] = transformer.transform( col_copy )

df [ numeric_cols ].head(5)



,loan_amount,applicant_income,population,ffiecmedian_family_income,number_of_owner-occupied_units,number_of_1_to_4_family_units
0,-0.531792,-0.714286,0.450973,-0.444174,-0.730861,0.881944
1,0.092486,-0.242857,0.293426,-0.813380,0.352871,0.352183
2,0.005780,-0.100000,0.410944,0.012356,-0.678230,-0.699405
3,-0.040462,0.442857,0.621373,0.698656,0.834928,0.541667
4,0.826590,-0.042857,0.645244,-0.284955,0.163876,0.093254


In [4]:
# category encoding 
import category_encoders as ce

#property_type_str  , loan_purpose_str  , preapproval_str  , applicant_race_str  , applicant_sex_str  

#drop_cols = [ 'occupancy_str', 'preapproval_str', 'applicant_ethnicity',
#                     'applicant_race', 'applicant_sex', 'county_code','occupancy']

drop_cols = [ 'loan_type_str', 'occupancy_str', 'applicant_ethnicity_str', 'msa_md_str', 'state_code_str', 'county_code_str' ]

df_clean = df.drop(columns=drop_cols)

ohe = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)
df = ohe.fit_transform(df_clean)
df.head()

,property_type_str_1 to 4 Family - non manufactured,property_type_str_Manufactured Housing,property_type_str_Multifamily,loan_purpose_str_Home purchase,loan_purpose_str_Refinancing,loan_purpose_str_Home improvement,preapproval_str_nan,preapproval_str_Requested,preapproval_str_Not Requested,applicant_race_str_White,...,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant,accepted
0,1,0,0,1,0,0,1,0,0,1,...,-0.714286,0.450973,0.44230,-0.444174,0.50933,-0.730861,0.881944,4536,0,1
1,1,0,0,0,1,0,1,0,0,1,...,-0.242857,0.293426,0.15905,-0.813380,1.00000,0.352871,0.352183,2458,0,0
2,1,0,0,0,1,0,1,0,0,1,...,-0.100000,0.410944,0.61270,0.012356,1.00000,-0.678230,-0.699405,5710,0,1
3,1,0,0,1,0,0,0,1,0,1,...,0.442857,0.621373,0.06246,0.698656,1.00000,0.834928,0.541667,5888,1,1
4,1,0,0,1,0,0,1,0,0,0,...,-0.042857,0.645244,1.00000,-0.284955,0.82200,0.163876,0.093254,289,0,1


In [5]:
# export
#df_clean = df.drop(columns=drop_cols)
df.to_csv('./export/feature_engineered.csv', index=False)

df.head(5)

,property_type_str_1 to 4 Family - non manufactured,property_type_str_Manufactured Housing,property_type_str_Multifamily,loan_purpose_str_Home purchase,loan_purpose_str_Refinancing,loan_purpose_str_Home improvement,preapproval_str_nan,preapproval_str_Requested,preapproval_str_Not Requested,applicant_race_str_White,...,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant,accepted
0,1,0,0,1,0,0,1,0,0,1,...,-0.714286,0.450973,0.44230,-0.444174,0.50933,-0.730861,0.881944,4536,0,1
1,1,0,0,0,1,0,1,0,0,1,...,-0.242857,0.293426,0.15905,-0.813380,1.00000,0.352871,0.352183,2458,0,0
2,1,0,0,0,1,0,1,0,0,1,...,-0.100000,0.410944,0.61270,0.012356,1.00000,-0.678230,-0.699405,5710,0,1
3,1,0,0,1,0,0,0,1,0,1,...,0.442857,0.621373,0.06246,0.698656,1.00000,0.834928,0.541667,5888,1,1
4,1,0,0,1,0,0,1,0,0,0,...,-0.042857,0.645244,1.00000,-0.284955,0.82200,0.163876,0.093254,289,0,1
